In [ ]:
from datetime import date
import random
import time
import yfinance as yf
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from numpy.fft import fft, ifft, fftshift
import numpy as np
from numpy import log, sqrt, exp


from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t, poisson
from scipy.linalg import solve_banded
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import quad
from scipy.special import roots_laguerre
from scipy.interpolate import interp1d
from scipy.sparse import diags, kron, identity, csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.stats import multivariate_normal, kstest

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

import pymc as pm
import arviz as az

#import aesara.tensor as at

from tensorflow import keras
#from tensorflow.keras.utils import plot_model

#import pyswarms as ps

######################################
#from pmdarima import auto_arima
#from diptest import diptest



In [ ]:
# Π ∼ N(Π,τΣ) and views Q ∼ N(Pμ,Ω)
class BlackLitterman:
    def __init__(self, cov_matrix, market_weights, risk_aversion, tau):
        self.Sigma = cov_matrix
        self.w_m = market_weights
        self.lambda_ = risk_aversion
        self.tau = tau
        self.trading_days = 252

        self.n_assets = len(market_weights)
        self.P = None
        self.Q = None
        self.Omega = None

    def implied_equilibrium_returns(self):
        """
        Compute the implied excess returns: Π = λ * Σ * w_m
        """
        return self.lambda_ * self.Sigma @ self.w_m

    def set_views(self, P, Q, Omega=None):
        """
        Set investor views.
        P: Pick matrix (K x N)
        Q: View vector (K x 1)
        Omega: Covariance of view errors (K x K)
        """
        self.P = np.atleast_2d(P)
        self.Q = np.atleast_2d(Q).reshape(-1, 1)

        if Omega is None:
            # Default: Omega = diag(P @ τΣ @ P.T) #By He Litterman
            self.Omega = np.diag(np.diag(self.P @ (self.tau * self.Sigma) @ self.P.T))
        else:
            self.Omega = Omega

    def posterior_returns(self):
        """
        Compute the Black-Litterman posterior expected returns.
        """
        Pi = self.implied_equilibrium_returns().reshape(-1, 1)
        tau_Sigma = self.tau * self.Sigma

        inv_term = np.linalg.inv(
            np.linalg.inv(tau_Sigma) + self.P.T @ np.linalg.inv(self.Omega) @ self.P
        )

        middle_term = (
            np.linalg.inv(tau_Sigma) @ Pi +
            self.P.T @ np.linalg.inv(self.Omega) @ self.Q
        )

        mu_bl = inv_term @ middle_term
        return mu_bl.flatten()

    def posterior_covariance(self):
        """
        Compute the Black-Litterman posterior covariance matrix.
        """
        tau_Sigma = self.tau * self.Sigma
        inv_term = np.linalg.inv(
            np.linalg.inv(tau_Sigma) + self.P.T @ np.linalg.inv(self.Omega) @ self.P
        )
        return self.Sigma + inv_term

    def optimal_weights(self, mu_bl):
        """
        Compute optimal weights using mean-variance optimization, the final result.
        """
        inv_Sigma = np.linalg.inv(self.Sigma)
        return (1 / self.lambda_) * inv_Sigma @ mu_bl

    def run(self):
        """
        Run full Black-Litterman model pipeline and return results.
        """
        mu_bl = self.posterior_returns()
        Sigma_bl = self.posterior_covariance()
        weights_bl = self.optimal_weights(mu_bl)
        return mu_bl, Sigma_bl, weights_bl
